## Newton polynomial interpolation

__MATH 240__ <br>
_spring 2021_ <br>

To start,  let's  build a function that evaluates a linear combination of Newton polynomials. The linear combination is
$$
    x \mapsto c_1 + c_2 (x - x_1) + c_3 (x-x_1)(x-x_2) +  c_4  (x-x_1)(x-x_2) (x-x_3) + \cdots
$$
The inputs will be the polynomial variable $x$, an array of coefficients $c$, and an array of the knots $x$. Our function uses a Horner-like scheme. Accordingly, we loop through the coefficients from high to low. The Julia way to loop backwards is to use `k = n:-1:1`

When the number of coefficients doesn't match the number of knots, let's issue an error.

In [ ]:
function newton_interpolation(x::Number,c::Array,knots::Array) 
   n = length(c)
   n == length(knots) || error("Oops")
   s = zero(x) #initialize accumulator to zero
   for k = n:-1:1
      s = c[k] + s*(x - knots[k])
    end
    s 
end

In [ ]:
using Gadfly

Let's test this; say our polynomial is

In [ ]:
P = x -> 46 + 107*(x-1) + 28 * (x-1)*(x-2)

The function $Q$ should exactly match the function $P$

In [ ]:
Q  = x -> newton_interpolation(x, [46.0,107.0,28.0],[1.0,2.0,3.0])

In [ ]:
D = x -> P(x) - Q(x)

In [ ]:
plot([P,Q],-20,20)

In [ ]:
plot(D,-20,20)

Here is a somewhat tricky function that finds the diagonal entries of the Newton divided difference table for a function $f$ with an array of knots $x$

In [ ]:
function newton_divided_diff(f::Function, x::Array)
    n = length(x)
    c = map(f,x)   
    for j in 2:n
         for i in n: -1 : j
            c[i]=(c[i]-c[i-1])/(x[i]-x[i-(j-1)])
        end
    end
    c
end

In [ ]:
newton_divided_diff(x -> 46 + 107(x-1) + 28 * (x-1)*(x-2), [1.0, 2.0,3.0])

Yes, it's true--the value of the last knot doesn't matter:

In [ ]:
newton_divided_diff(x -> 46 + 107*(x-1) + 28 * (x-1)*(x-2), [1.0, 2.0,1932.0])

Let's interpolate the sine function; we'll use six equally spaced knots in the interval $[-\pi, \pi]$

In [ ]:
N = 5

In [ ]:
knots = [-pi + 2*pi*k/N for k in 0:N]

In [ ]:
c = newton_divided_diff(x -> sin(x), knots)

In [ ]:
plot([x -> newton_interpolation(x,c,knots), x -> sin(x)], -pi,pi)

In [ ]:
plot([x -> newton_interpolation(x,c,knots) - sin(x)], -pi,pi)

Let's try something wild and crazy. We'll interpolate the factorial function.  Actually, let's try the reciprocal of the factorial function. We must use knots that are integers

In [ ]:
knots = [k for k in 0 : 20];

In [ ]:
function myfac(x)
    convert(Float64, 1/factorial(x))
end

Pretty cool--the Newton polynomial coefficients appear to diminish toward zero.

In [ ]:
c = newton_divided_diff(myfac, knots)

In [ ]:
PP = x -> newton_interpolation(x,c,knots) 

In [ ]:
plot(PP,0,20)

The gamma function interpolates the factorial function, effectively extending the factorial function to the real line. Actually, for $k \in \mathbf{Z}_{> -1}$, we have
$$
  \Gamma(k) = (k-1)!
$$
Julia's gamma function is in a package `SpecialFunctions`

In [ ]:
using Pkg

In [ ]:
Pkg.add("SpecialFunctions")

In [ ]:
using SpecialFunctions

In [ ]:
plot([PP, x -> 1/gamma(x+1)],0,20)

In [ ]:
D = x -> PP(x) - 1/gamma(x+1)

In [ ]:
plot(D,0,20)